In [1]:
import pandas as pd
import codecs
import json
from pandas.io.json import json_normalize

In [2]:
#Machine learning libraries for Text analysis
import nltk  # machine learning with text
from nltk import download as nltk_download
from nltk.corpus import stopwords, wordnet  # List of common words
from nltk.tokenize import word_tokenize  # Split text into significant forms
from nltk.stem import WordNetLemmatizer  # Reduce words to their root form
from nltk import pos_tag  # Tag words with parts of speech
from collections import defaultdict  # Dictionaries that have a backup value
from sklearn.feature_extraction.text import CountVectorizer  # Convert text to sparse matrices
from textblob import TextBlob  # Sentiment analysis
from gensim.models.ldamodel import LdaModel  # Topic extraction
from gensim.models.phrases import Phrases  # ngrams
from gensim import corpora  # Vectorization
import pyLDAvis

In [ ]:
from lyricsgenius import Genius

genius = Genius('JOSfSO0Cn0Y3avlw_ItRLdmwFj3s-36fjS-POUolMGFiWkUiE_zHXfQZVsOOqbxA')
artist = genius.search_artist("Kendrick Lamar", max_songs=200)
kendrick_json = artist.save_lyrics()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Searching for songs by Kendrick Lamar...

Song 1: "HUMBLE."
Song 2: "​m.A.A.d city"


In [ ]:
#kendrick = pd.read_json('Lyrics_kendrickEilish.json', orient='index')
#kendrick = pd.read_json(codecs.open('Lyrics_kendrickEilish.json','r','utf-8'))
#df = json.loads('Lyrics_kendrickEilish.json').text
#df = pd.DataFrame.from_dict('Lyrics_kendrickEilish.json', orient='index')
with open('Lyrics_KendrickLamar.json') as json_data:
    data = json.load(json_data)

df = pd.DataFrame(data['songs'])
kendrick_df = df[['full_title', 'artist', 'lyrics']]

In [ ]:
kendrick_df.lyrics[1]

# Lyric Analysis

## Cleaning the lyrics

In [ ]:
kendrick_df.dtypes

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.str.lower()

In [ ]:
# Remove quotes
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="(\u2019)", value="'", regex=True)

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="won’t", value="will not", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="can’t", value="can not", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="wanna", value="want to", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="n't", value=" not", regex=True)

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="it's", value="it is", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="i've", value="i have", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="i'm", value="i am", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace="n' ", value="ng ", regex=True)

In [ ]:
# remove "[verse ..]" and "[chorus]"
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=r"\[verse \d\]", value="", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=r"\[chorus\]", value="", regex=True)
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=r"\[bridge\]", value="", regex=True)

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=" ", regex=True)

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=r"\[chorus\]", value="", regex=True)

In [ ]:
kendrick_df.lyrics = kendrick_df.lyrics.replace(to_replace=r"\W", value=" ", regex=True)

In [ ]:
# Remove common words from English lyrics 

kendrick_df.lyrics = kendrick_df.lyrics.apply(word_tokenize)
stop_words = stopwords.words("english")
kendrick_df.lyrics = kendrick_df.lyrics.apply(lambda x: [y for y in x if y not in stop_words
                                                              and len(y) > 2])

In [ ]:
kendrick_df.lyrics[1]

In [ ]:
lemma = WordNetLemmatizer()

# Dictionary to convert between part-of-speech tags and the ones the lemmatizer understands

tag_dict = defaultdict(lambda : wordnet.NOUN)
tag_dict['J'] = wordnet.ADJ
tag_dict['V'] = wordnet.VERB
tag_dict['R'] = wordnet.ADV

# function to lemmatize a sentence

def get_lemma(word):
    """Gets the POS tag for a word, and then returns the lemmatized form of the word"""
    tag = pos_tag([word])[0][1][0]
    tag = tag_dict[tag]
    
    return lemma.lemmatize(word, tag)

# Perform the lemmatization
kendrick_df.lyrics = kendrick_df.lyrics.apply(lambda x: [get_lemma(word) for word in x])

In [ ]:
# Create the list of words and their frequency (corpus)
from gensim import corpora
pyLDAvis.enable_notebook()
dictionary = corpora.Dictionary(kendrick_df.lyrics )
corpus = [dictionary.doc2bow(text) for text in kendrick_df.lyrics ]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(lda_display)